In [1]:
#!g2.1
# %pip install optimum
# %pip install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
# %pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7
    
# %pip install langchain --upgrade
# %pip install langchain_experimental --upgrade

# %pip install python-dotenv

# !mkdir Mistral-7B-Instruct-v0.1-GPTQ
# !git clone --single-branch --branch gptq-4bit-32g-actorder_True https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GPTQ

In [2]:
#!g2.1
few_shots_string = """
<s>[INST] You are a helpful bot.
You are given the question, the context that contains the answer to the question, and a Database Schema.
Your task is to translate a question into a PostgreSQL query based on the context and the Database Schema, with interleaving Thought and Response steps.
Your goal is not to answer the question but to create a PostgreSQL query, that is going to select the necessary data that will help to answer the question.
Please wrap your code answer using ```

In order to complete the task, follow the guidelines:
Question: [QUESTION]
Context: [CONTEXT]
Database Schema: [DATABASE SCHEMA]
###
Thought: I need to read the given question and context, and decide, what information might help to answer the question.
Response: [BOT RESPONSE]
###
Thought: I know what information might help to answer the question. Now I need to get features in the provided Database Schema, that correspond to the information that might help to answer the question.
Response: [BOT RESPONSE]
###
Thought: I know what information is needed, I know what features in the Database correspond to this information and now I need to decide, what I need to Select from the database
Response: [BOT RESPONSE]
###
Thought: Now I understand what I need to Select from the database, here is the PostgreSQL query:
Response: [BOT RESPONSE] 

Here is an example of how you should act:

Question: What is my ideal weight? My ID is 212
Context: Adults (26 to 65 years) should sleep from 7 to 9 hours. Teenagers (17 to 25) should sleep from 8 to 10 hours. * For men:
+ Under 5'5" (165 cm): 130-154 lbs (59-70 kg)
+ 5'5"-5'9" (165-175 cm): 140-163 lbs (64-74 kg)
+ 6'0"-6'3" (183-191 cm): 154-181 lbs (70-83 kg)
+ Over 6'3" (191 cm): 163-190 lbs (74-86 kg)
* For women:
+ Under 5'5" (165 cm): 115-139 lbs (52-63 kg)
+ 5'5"-5'9" (165-175 cm): 125-147 lbs (57-66 kg)
Database Schema: CREATE TABLE user_info (id INT PRIMARY KEY, age INT, gender VARCHAR(10), height FLOAT, weight FLOAT, sleep_duration_hours FLOAT, sleep_quality_rating INT, bedtime TIMESTAMP, wake_up_time TIMESTAMP);
###
Thought: I need to read the given question and context, and decide, what information might help to answer the question.
Response: It seems that the ideal weight for a person depends on gender, height, and weight.
###
Thought: I know what information might help to answer the question. Now I need to get features in the provided Database Schema, that correspond to the information that might help to answer the question.
Response: Feature 'gender' in the table, user_info corresponds to a person's gender, feature 'height' corresponds to a person's height, and feature 'weight' corresponds to a person's weight
###
Thought: I know what information is needed, I know what features in the Database correspond to this information and now I need to decide, what I need to Select from the database
Response: I need to select 'gender', 'height', and 'weight' features for a person with ID=212
###
Thought: Now I understand what I need to Select from the database, I must write a PostgreSQL query.
Response: SELECT gender, height, weight FROM user_info WHERE id = 212; [/INST]

Question: {question}
Context: {context}
Database Schema: {table}
###1
"""

In [3]:
#!g2.1
from sqlalchemy import create_engine, text, inspect, types
import pandas as pd
from decimal import Decimal
from dotenv import load_dotenv
import os

load_dotenv()

# PostgreSQL connection parameters
params = {
    'dbname': 'pagila',
    'user': 'postgres',
    'password': os.environ.get("postgres_password"),
    'host': os.environ.get("postgres_host"),
    'port': 5432
}

# Create an SQLAlchemy engine

db_url = 'postgresql://{}:{}@{}:{}/{}'.format(
    params['user'],
    params['password'],
    params['host'],
    params['port'],
    params['dbname']
)

postgres_engine = create_engine(db_url)

# Get list of all tables in PostgreSQL database
inspector = inspect(postgres_engine)
inspector.get_table_names()

['film_actor',
 'address',
 'city',
 'actor',
 'category',
 'country',
 'customer',
 'film',
 'film_category',
 'inventory',
 'language',
 'rental',
 'staff',
 'payment',
 'store',
 'payment_p2022_07',
 'payment_p2022_02',
 'payment_p2022_03',
 'payment_p2022_04',
 'payment_p2022_05',
 'payment_p2022_01',
 'payment_p2022_06']

In [4]:
#!g2.1
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.schema import CreateTable
import psycopg2

def get_create_table_statement(engine, table_name):
    """
    Get the CREATE TABLE statement for a specific table from the PostgreSQL database using SQLAlchemy.

    Parameters:
        engine (sqlalchemy.engine.Engine): The SQLAlchemy engine object.
        table_name (str): The name of the table.

    Returns:
        str: The CREATE TABLE statement, or None if the table does not exist.
    """
    metadata = MetaData()
    metadata.bind = engine
    
    # Reflect the table
    try:
        table = Table(table_name, metadata, autoload_with=engine)
    except (KeyError, psycopg2.errors.UndefinedTable):
        # Table does not exist
        return None
    
    # Get the CREATE TABLE statement
    create_table_statement = str(CreateTable(table).compile(engine))
    
    return create_table_statement.replace("\n", "").replace("\t", "")

get_create_table_statement(postgres_engine, "actor")

'CREATE TABLE actor (actor_id SERIAL NOT NULL, first_name TEXT NOT NULL, last_name TEXT NOT NULL, last_update TIMESTAMP WITH TIME ZONE DEFAULT now() NOT NULL, CONSTRAINT actor_pkey PRIMARY KEY (actor_id))'

In [5]:
#!g2.1
from langchain.utilities import SQLDatabase
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_experimental.sql import SQLDatabaseChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GPTQConfig

model_name_or_path = "Mistral-7B-local"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

quantization_config = GPTQConfig(bits=4, tokenizer=tokenizer, disable_exllama=True)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main",
                                             quantization_config=quantization_config)

pipe = pipeline(
   "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

db = SQLDatabase(postgres_engine, include_tables=["film", "actor"])

llm = HuggingFacePipeline(pipeline=pipe)

prompt = PromptTemplate.from_template(few_shots_string)

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

2023-10-11 01:26:58.702588: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [6]:
#!g2.1

%%time
db_chain(prompt.format(question="How many actors are there?",
                       context="Actors play in films",
                       table=get_create_table_statement(postgres_engine, "actor")))["result"]



> Entering new SQLDatabaseChain chain...

<s>[INST] You are a helpful bot.
You are given the question, the context that contains the answer to the question, and a Database Schema.
Your task is to translate a question into a PostgreSQL query based on the context and the Database Schema, with interleaving Thought and Response steps.
Your goal is not to answer the question but to create a PostgreSQL query, that is going to select the necessary data that will help to answer the question.
Please wrap your code answer using ```

In order to complete the task, follow the guidelines:
Question: [QUESTION]
Context: [CONTEXT]
Database Schema: [DATABASE SCHEMA]
###
Thought: I need to read the given question and context, and decide, what information might help to answer the question.
Response: [BOT RESPONSE]
###
Thought: I know what information might help to answer the question. Now I need to get features in the provided Database Schema, that correspond to the information that might help to answe

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


SELECT COUNT(*) FROM actor;
SQLResult: [(200,)]
Answer:There are 200 actors in the database.
###
> Finished chain.
CPU times: user 5.16 s, sys: 434 ms, total: 5.6 s
Wall time: 6.43 s


'There are 200 actors in the database.\n###'

In [7]:
#!g2.1

%%time
db_chain(prompt.format(question="How many movies came out in 2006?",
                       context="Actors play in films",
                       table=get_create_table_statement(postgres_engine, "film")))["result"]



> Entering new SQLDatabaseChain chain...

<s>[INST] You are a helpful bot.
You are given the question, the context that contains the answer to the question, and a Database Schema.
Your task is to translate a question into a PostgreSQL query based on the context and the Database Schema, with interleaving Thought and Response steps.
Your goal is not to answer the question but to create a PostgreSQL query, that is going to select the necessary data that will help to answer the question.
Please wrap your code answer using ```

In order to complete the task, follow the guidelines:
Question: [QUESTION]
Context: [CONTEXT]
Database Schema: [DATABASE SCHEMA]
###
Thought: I need to read the given question and context, and decide, what information might help to answer the question.
Response: [BOT RESPONSE]
###
Thought: I know what information might help to answer the question. Now I need to get features in the provided Database Schema, that correspond to the information that might help to answe

'1000 movies came out in 2006'

In [8]:
#!g2.1


In [ ]:
#!g2.1
